In [11]:
# Install the required packages
! pip install -q langchain langchain-ibm pydantic python-dotenv rich

In [12]:
from langchain_ibm import ChatWatsonx
from dotenv import load_dotenv
from pydantic import BaseModel
from rich import print
import os
# Load environment variables from .env file
# Ensure you have a .env file with the following variables:
# WATSONX_APIKEY, WATSONX_PROJECT_ID, WATSONX_URL
load_dotenv()

True

In [13]:
invoice_data = """
ACME Ltd. – Tax ID 76,543,210-K
123 Main Street, Anytown | Phone: +1 (555) 123-4567
Contact: billing@acme.com

ELECTRONIC INVOICE
Invoice No.: 2025-0042
Issue Date: May 15, 2025
Purchase Order: PO-984-2025

DETAILS
	•	10 × M6 × 30 mm screws … USD 0.15 each … USD 1.50
	•	5 × 4 mm Allen keys … USD 1.20 each … USD 6.00
	•	3 × 200 ml multi-purpose lubricant … USD 3.50 each … USD 10.50

Subtotal: USD 18.00
VAT (19%): USD 3.42
Total: USD 21.42

Currency: USD
Due Date: June 10, 2025
Payment Method: Bank transfer (30 days)
Account No.: 012-34567-89 | International Bank

Notes:
Please deliver to the central warehouse by May 20, 2025.
"""

prompt = f"""
You are an intelligent agent responsible for extracting information from invoices.
Read the invoice and look for the requested information.
The invoice may contain one or more products; make sure to extract the information for each one in the order in which they appear.
The invoice is as follows:
{invoice_data}
"""

In [ ]:
class InvoiceItem(BaseModel):
    name: str
    quantity: int
    unit_price: float

class Invoice(BaseModel):
    purchase_order: str
    invoice_date: str
    subtotal: float
    tax: float
    total: float
    items: list[InvoiceItem]
    

In [ ]:
llm = ChatWatsonx(
    model_id="ibm/granite-3-3-8b-instruct",
    project_id=os.getenv("WATSONX_PROJECT_ID"),
)

In [15]:
result = llm.with_structured_output(Invoice).invoke(prompt)
print(result)

Invoice(
    purchase_order='PO-984-2025',
    invoice_date='May 15, 2025',
    subtotal=18.0,
    tax=3.42,
    total=21.42,
    items=[
        InvoiceItem(name='M6 × 30 mm screws', quantity=10, unit_price=0.15),
        InvoiceItem(name='4 mm Allen keys', quantity=5, unit_price=1.2),
        InvoiceItem(name='200 ml multi-purpose lubricant', quantity=3, unit_price=3.5)
    ]
)